In [5]:
from sklearn.preprocessing import MinMaxScaler
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import NaiveSeasonal, NaiveDrift, AutoARIMA, ARIMA,XGBModel,Prophet
import numpy as np
from darts.metrics import mape, mase, mae, rmse
import matplotlib.pyplot as plt
import os
import pandas as pd


/home/apprenant/miniconda3/envs/data_analyse_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:

df_tel = pd.read_csv('telephone.csv', index_col="date_appel", parse_dates=True)
df_tel["nombre_entrants_corrige"] = df_tel["nombre_entrants_corrige"].str.replace(',', '.').astype(float)

In [26]:

print(df_tel.head())
print(df_tel.shape)
liste_famille = df_tel.famille.unique()
liste_entite = df_tel.entite.unique()
taille_entite_famille = df_tel[(df_tel["entite"] == 'Entite 2') & (df_tel["famille"] == 'F5')].shape[0]
print("Taille de l'entité-famille :", taille_entite_famille)
df_naive_saisonal = pd.DataFrame(columns=['model', 'entite', 'famille', 'mape', 'mase', 'mae', 'rmse'])

for i, famille in enumerate(liste_famille):
    for j, entite in enumerate(liste_entite):
        print(entite, famille)

        df = df_tel[(df_tel["entite"] == entite) & (df_tel["famille"] == famille)]
        if len(df) > 28:
            df = df.asfreq('d')
            df.entite = df.entite.fillna(entite)
            df.famille = df.famille.fillna(famille)
            df.nombre_entrants_corrige = df.nombre_entrants_corrige.fillna(0)
            df = df.reset_index(names='date')
            series = TimeSeries.from_dataframe(df=df, time_col="date", value_cols='nombre_entrants_corrige')
            scaler = MinMaxScaler(feature_range=(-1, 1))
            scaler = Scaler(scaler)

            train, val = series[:-45], series[-45:]
            train = scaler.fit_transform(train)
            val = scaler.transform(val)
            series_norm = scaler.transform(series)
            model = NaiveSeasonal(K=7)  # init
            model.fit(train)  # fit
            pred = model.predict(n=45)
            mape_score = mape(val, pred)
            mase_score = mase(val, pred, insample=train, m=7)
            mae_score = mae(val, pred)
            rmse_score = rmse(val, pred)
            row_dict = {
                'model': 'NaiveSeasonal',
                'entite': entite, 'famille': famille,
                'mape': mape_score, 'mase': mase_score,
                'mae': mae_score, 'rmse': rmse_score
            }
            df_naive_saisonal = pd.concat([df_naive_saisonal, pd.DataFrame([row_dict])], ignore_index=True)

            plt.figure(figsize=(8, 6))
            series_norm[-90:].plot(lw=3)
            pred.plot(lw=3, label='forecast')
            plt.savefig(f'images/naive_saisonal/{entite}_{famille}.png')
# Créer le répertoire "data" s'il n'existe pas déjà
if not os.path.exists("data"):
    os.makedirs("data")
df_naive_saisonal.to_csv("data/df_naive_saisonal.csv", index=False)


            id    entite famille  nombre_entrants_corrige
date_appel                                               
2019-01-02  12  Entite 1      F2                    165.0
2019-01-02  13  Entite 1      F3                     54.0
2019-01-02  14  Entite 1      F4                      1.0
2019-01-02  15  Entite 1      F5                     99.0
2019-01-02  16  Entite 1      F6                   1487.0
(23783, 4)
Taille de l'entité-famille : 28
Entite 1 F2
Entite 2 F2
Entite 3 F2
Entite 4 F2
Entite 5 F2
Entite 1 F3
Entite 2 F3
Entite 3 F3
Entite 4 F3
Entite 5 F3
Entite 1 F4
Entite 2 F4
Entite 3 F4
Entite 4 F4
Entite 5 F4
Entite 1 F5
Entite 2 F5
Entite 3 F5
Entite 4 F5
Entite 5 F5
Entite 1 F6
Entite 2 F6
Entite 3 F6
Entite 4 F6
Entite 5 F6
Entite 1 F7
Entite 2 F7
Entite 3 F7
Entite 4 F7
Entite 5 F7
Entite 1 F1
Entite 2 F1
Entite 3 F1
Entite 4 F1


/tmp/ipykernel_6832/553690872.py:46: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(8, 6))


Entite 5 F1


In [53]:
%matplotlib agg

print(df_tel.shape)
liste_famille = df_tel.famille.unique()
liste_entite = df_tel.entite.unique()

df_result = pd.DataFrame(columns = ['model','entite', 'famille','mape','mase','mae','mse'])
model_name = 'auto_arima'
os.mkdir(f'images/output/{model_name}')
for i, famille in enumerate(liste_famille):
    for j, entite in enumerate(liste_entite):
        print(entite,famille)
        
        df = df_tel[(df_tel["entite"] == entite)& (df_tel["famille"] == famille)]  
        if len(df)>28:      
            df = df.asfreq('d')
            df.entite = df.entite.fillna(entite)
            df.famille = df.famille.fillna(famille)
            df.nombre_entrants_corrige = df.nombre_entrants_corrige.fillna(0)
            df = df.reset_index(names='date')
            series = TimeSeries.from_dataframe(df=df, time_col="date", value_cols='nombre_entrants_corrige')
            scaler = MinMaxScaler(feature_range=(-1, 1))
            scaler = Scaler(scaler)

            train, val = series[:-45], series[-45:]
            train = scaler.fit_transform(train)
            val = scaler.transform(val)
            series_norm = scaler.transform(series)
            model = AutoARIMA( start_p=1, start_q=1,
                         test='adf',
                         max_p=3, max_q=3, m=7,
                         start_P=0, seasonal=True,
                         d=None, # it means use the test
                         D=1, trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)           # init
            model.fit(train)                      # fit
            pred = model.predict(n=45) 
            mape_score = mape(val, pred)
            mase_score = mase(val, pred,insample = train, m=7)
            mae_score = mae(val, pred)
            rmse_score = rmse(val, pred)
            row_dict= {
                'model':'NaiveDrift',
                'entite':entite, 'famille':famille,
                'mape':mape_score, 'mase':mase_score,
                'mae':mae_score, 'rmse':rmse_score
            }
            df_result = df_result.append(row_dict, ignore_index=True)
            


            plt.figure(figsize=(8,6))
            series_norm[-90:].plot(lw=3)
            pred.plot(lw=3, label='forecast')
            plt.savefig(f'images/output/{model_name}/{entite}_{famille}.png')
df_result.to_csv(f"data/{model_name}.csv",index=False)

(23783, 4)
Entite 1 F2
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,1,1)[7] intercept   : AIC=-1658.014, Time=5.68 sec
 ARIMA(0,0,0)(0,1,0)[7] intercept   : AIC=-1092.810, Time=0.35 sec
 ARIMA(1,0,0)(1,1,0)[7] intercept   : AIC=-1436.595, Time=1.65 sec
 ARIMA(0,0,1)(0,1,1)[7] intercept   : AIC=-1509.955, Time=1.28 sec
 ARIMA(0,0,0)(0,1,0)[7]             : AIC=-1094.752, Time=0.14 sec
 ARIMA(1,0,1)(0,1,0)[7] intercept   : AIC=-1139.611, Time=0.71 sec
 ARIMA(1,0,1)(1,1,1)[7] intercept   : AIC=inf, Time=3.54 sec
 ARIMA(1,0,1)(0,1,2)[7] intercept   : AIC=inf, Time=4.49 sec
 ARIMA(1,0,1)(1,1,0)[7] intercept   : AIC=-1471.410, Time=2.46 sec
 ARIMA(1,0,1)(1,1,2)[7] intercept   : AIC=-1693.068, Time=7.38 sec
 ARIMA(1,0,1)(2,1,2)[7] intercept   : AIC=inf, Time=7.86 sec
 ARIMA(1,0,1)(2,1,1)[7] intercept   : AIC=inf, Time=7.07 sec
 ARIMA(0,0,1)(1,1,2)[7] intercept   : AIC=inf, Time=5.66 sec
 ARIMA(1,0,0)(1,1,2)[7] intercept   : AIC=inf, Time=5.61 sec
 ARIMA(2,0,1)(1,1,2)[7] intercep

/tmp/ipykernel_6832/1064708107.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(row_dict, ignore_index=True)


Entite 2 F2
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,1,1)[7] intercept   : AIC=-605.242, Time=4.12 sec
 ARIMA(0,0,0)(0,1,0)[7] intercept   : AIC=12.508, Time=0.26 sec
 ARIMA(1,0,0)(1,1,0)[7] intercept   : AIC=-267.328, Time=0.86 sec
 ARIMA(0,0,1)(0,1,1)[7] intercept   : AIC=-478.213, Time=1.25 sec
 ARIMA(0,0,0)(0,1,0)[7]             : AIC=10.612, Time=0.15 sec
 ARIMA(1,0,1)(0,1,0)[7] intercept   : AIC=-12.453, Time=0.76 sec
 ARIMA(1,0,1)(1,1,1)[7] intercept   : AIC=inf, Time=3.55 sec
 ARIMA(1,0,1)(0,1,2)[7] intercept   : AIC=inf, Time=5.46 sec
 ARIMA(1,0,1)(1,1,0)[7] intercept   : AIC=-279.269, Time=1.53 sec
 ARIMA(1,0,1)(1,1,2)[7] intercept   : AIC=-612.357, Time=6.57 sec
 ARIMA(1,0,1)(2,1,2)[7] intercept   : AIC=inf, Time=6.68 sec
 ARIMA(1,0,1)(2,1,1)[7] intercept   : AIC=inf, Time=6.77 sec
 ARIMA(0,0,1)(1,1,2)[7] intercept   : AIC=inf, Time=4.83 sec
 ARIMA(1,0,0)(1,1,2)[7] intercept   : AIC=inf, Time=4.62 sec
 ARIMA(2,0,1)(1,1,2)[7] intercept   : AIC=-624.785, Time

/tmp/ipykernel_6832/1064708107.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(row_dict, ignore_index=True)


Entite 3 F2
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,1,1)[7] intercept   : AIC=-1219.922, Time=2.33 sec
 ARIMA(0,0,0)(0,1,0)[7] intercept   : AIC=-446.964, Time=0.25 sec
 ARIMA(1,0,0)(1,1,0)[7] intercept   : AIC=-832.915, Time=0.91 sec
 ARIMA(0,0,1)(0,1,1)[7] intercept   : AIC=-1180.184, Time=1.21 sec
 ARIMA(0,0,0)(0,1,0)[7]             : AIC=-448.810, Time=0.13 sec
 ARIMA(1,0,1)(0,1,0)[7] intercept   : AIC=-468.628, Time=0.63 sec
 ARIMA(1,0,1)(1,1,1)[7] intercept   : AIC=-1217.897, Time=2.76 sec
 ARIMA(1,0,1)(0,1,2)[7] intercept   : AIC=-1217.061, Time=4.70 sec
 ARIMA(1,0,1)(1,1,0)[7] intercept   : AIC=-847.392, Time=1.32 sec
 ARIMA(1,0,1)(1,1,2)[7] intercept   : AIC=-1217.129, Time=5.36 sec
 ARIMA(1,0,0)(0,1,1)[7] intercept   : AIC=-1186.104, Time=1.05 sec
 ARIMA(2,0,1)(0,1,1)[7] intercept   : AIC=-1218.164, Time=2.89 sec
 ARIMA(1,0,2)(0,1,1)[7] intercept   : AIC=-1217.748, Time=3.28 sec
 ARIMA(0,0,0)(0,1,1)[7] intercept   : AIC=-1157.535, Time=0.65 sec
 ARIMA(0,0,2

...

/tmp/ipykernel_6832/1064708107.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(row_dict, ignore_index=True)


Entite 3 F1
Entite 4 F1
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,1,1)[7] intercept   : AIC=inf, Time=2.32 sec
 ARIMA(0,0,0)(0,1,0)[7] intercept   : AIC=-992.742, Time=0.14 sec
 ARIMA(1,0,0)(1,1,0)[7] intercept   : AIC=-1371.737, Time=0.81 sec
 ARIMA(0,0,1)(0,1,1)[7] intercept   : AIC=inf, Time=1.79 sec
 ARIMA(0,0,0)(0,1,0)[7]             : AIC=-994.719, Time=0.08 sec
 ARIMA(1,0,0)(0,1,0)[7] intercept   : AIC=-991.493, Time=0.20 sec
 ARIMA(1,0,0)(2,1,0)[7] intercept   : AIC=-1477.483, Time=2.34 sec
 ARIMA(1,0,0)(2,1,1)[7] intercept   : AIC=inf, Time=5.05 sec
 ARIMA(1,0,0)(1,1,1)[7] intercept   : AIC=inf, Time=4.28 sec
 ARIMA(0,0,0)(2,1,0)[7] intercept   : AIC=-1478.296, Time=1.23 sec
 ARIMA(0,0,0)(1,1,0)[7] intercept   : AIC=-1371.677, Time=0.73 sec
 ARIMA(0,0,0)(2,1,1)[7] intercept   : AIC=inf, Time=3.55 sec
 ARIMA(0,0,0)(1,1,1)[7] intercept   : AIC=inf, Time=1.55 sec
 ARIMA(0,0,1)(2,1,0)[7] intercept   : AIC=-1477.573, Time=1.61 sec
 ARIMA(1,0,1)(2,1,0)[7] intercept 

/tmp/ipykernel_6832/1064708107.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(row_dict, ignore_index=True)
/tmp/ipykernel_6832/1064708107.py:54: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(8,6))


Entite 5 F1
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,1,1)[7] intercept   : AIC=-1425.980, Time=2.75 sec
 ARIMA(0,0,0)(0,1,0)[7] intercept   : AIC=-669.034, Time=0.32 sec
 ARIMA(1,0,0)(1,1,0)[7] intercept   : AIC=-1049.644, Time=0.88 sec
 ARIMA(0,0,1)(0,1,1)[7] intercept   : AIC=-1382.693, Time=1.43 sec
 ARIMA(0,0,0)(0,1,0)[7]             : AIC=-671.033, Time=0.11 sec
 ARIMA(1,0,1)(0,1,0)[7] intercept   : AIC=-770.650, Time=0.58 sec
 ARIMA(1,0,1)(1,1,1)[7] intercept   : AIC=-1440.180, Time=3.44 sec
 ARIMA(1,0,1)(1,1,0)[7] intercept   : AIC=-1053.647, Time=1.68 sec
 ARIMA(1,0,1)(2,1,1)[7] intercept   : AIC=-1438.845, Time=5.72 sec
 ARIMA(1,0,1)(1,1,2)[7] intercept   : AIC=-1435.738, Time=5.67 sec
 ARIMA(1,0,1)(0,1,2)[7] intercept   : AIC=-1441.145, Time=5.34 sec
 ARIMA(0,0,1)(0,1,2)[7] intercept   : AIC=-1399.312, Time=3.75 sec
 ARIMA(1,0,0)(0,1,2)[7] intercept   : AIC=-1426.194, Time=3.94 sec
 ARIMA(2,0,1)(0,1,2)[7] intercept   : AIC=-1444.003, Time=6.80 sec
 ARIMA(2,0

/tmp/ipykernel_6832/1064708107.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(row_dict, ignore_index=True)


In [62]:
import os
import matplotlib.pyplot as plt
%matplotlib agg

sarima_parameters = pd.read_csv('sarima_parameters.csv')

print(df_tel.shape)
liste_famille = df_tel.famille.unique()
liste_entite = df_tel.entite.unique()

df_result = pd.DataFrame(columns=['model', 'entite', 'famille', 'mape', 'mase', 'mae', 'rmse'])
model_name = 'sarima'
os.makedirs(f'images/output/{model_name}', exist_ok=True)

for i, famille in enumerate(liste_famille):
    for j, entite in enumerate(liste_entite):
        print(entite, famille)

        df = df_tel[(df_tel["entite"] == entite) & (df_tel["famille"] == famille)]

        sarima_parameter = sarima_parameters[(sarima_parameters["Entite"] == entite) & (sarima_parameters["Famille"] == famille)]
        if len(df) > 0 and not sarima_parameter.empty:
            df = df.asfreq('d')
            df['Entite'] = df['entite'].fillna(entite)
            df['famille'] = df['famille'].fillna(famille)
            df['nombre_entrants_corrige'] = df['nombre_entrants_corrige'].fillna(0)
           
            df = df.reset_index(names='date')
            series = TimeSeries.from_dataframe(df=df, time_col="date", value_cols='nombre_entrants_corrige')
            
            scaler = MinMaxScaler(feature_range=(-1, 1))
            scaler = Scaler(scaler)

            train, val = series[:-45], series[-45:]
            train = scaler.fit_transform(train)
            val = scaler.transform(val)
            series_norm = scaler.transform(series)
            print(sarima_parameter.p)
            model = ARIMA(
                p=int(sarima_parameter.p.iloc[0]),
                q=int(sarima_parameter.q.iloc[0]),
                d=int(sarima_parameter.d.iloc[0]),
                seasonal_order=(
                    int(sarima_parameter.P),
                    int(sarima_parameter.D),
                    int(sarima_parameter.Q),
                    int(sarima_parameter.m)
                )
            )
            model.fit(train)
            pred = model.predict(n=45)

            mape_score = mape(val, pred)
            mase_score = mase(val, pred, insample=train, m=7)
            mae_score = mae(val, pred)
            rmse_score = rmse(val, pred)

            row_dict = {
                'model': 'Sarima',
                'entite': entite,
                'famille': famille,
                'mape': mape_score,
                'mase': mase_score,
                'mae': mae_score,
                'rmse': rmse_score
            }
            df_result = df_result.append(row_dict, ignore_index=True)

            plt.figure(figsize=(8, 6))
            series_norm[-90:].plot(lw=3)
            pred.plot(lw=3, label='forecast')
            plt.savefig(f'images/output/{model_name}/{entite}_{famille}.png')

df_result.to_csv(f"data/{model_name}.csv",index=False)

(23783, 4)
Entite 1 F2
3    0
Name: p, dtype: int64


/tmp/ipykernel_4949/1285259228.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(row_dict, ignore_index=True)


Entite 2 F2
4    2
Name: p, dtype: int64


/tmp/ipykernel_4949/1285259228.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(row_dict, ignore_index=True)


Entite 3 F2
5    0
Name: p, dtype: int64


...

/tmp/ipykernel_4949/1285259228.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(row_dict, ignore_index=True)


Entite 2 F1
1    1
Name: p, dtype: int64


/tmp/ipykernel_4949/1285259228.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(row_dict, ignore_index=True)


Entite 3 F1
Entite 4 F1
Entite 5 F1
2    1
Name: p, dtype: int64


/tmp/ipykernel_4949/1285259228.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_result = df_result.append(row_dict, ignore_index=True)


In [64]:
df_auto_arima = pd.read_csv("/home/apprenant/Documents/timesseries/data/auto_arima.csv")
df_naive_saisonal = pd.read_csv("/home/apprenant/Documents/timesseries/data/df_naive_saisonal.csv")
df_sarima = pd.read_csv("/home/apprenant/Documents/timesseries/data/sarima.csv")
df_auto_arima = df_auto_arima.drop(["mae", "mse","mase"], axis=1)
df_naive_saisonal = df_naive_saisonal.drop(["mae", "mse","mase"], axis=1)
df_sarima = df_sarima.drop(["mae","mase"], axis=1)

In [65]:
df_sarima

,model,entite,famille,mape,rmse
0,Sarima,Entite 1,F2,6.782866,0.068331
1,Sarima,Entite 2,F2,25.293153,0.106689
2,Sarima,Entite 3,F2,13.002076,0.148313
3,Sarima,Entite 5,F2,8.478303,0.061541
4,Sarima,Entite 1,F3,390.206353,0.209340
5,Sarima,Entite 2,F3,45.078620,0.155179
6,Sarima,Entite 5,F3,380.610118,0.223811
7,Sarima,Entite 1,F5,11.945766,0.102069
8,Sarima,Entite 3,F5,24.805123,0.164567
9,Sarima,Entite 4,F5,20.686650,0.160283


In [66]:
df_naive_saisonal

,model,entite,famille,mape,rmse
0,NaiveSeasonal,Entite 1,F2,2.844508,0.033093
1,NaiveSeasonal,Entite 2,F2,35.387882,0.145590
2,NaiveSeasonal,Entite 3,F2,16.542492,0.191317
3,NaiveSeasonal,Entite 5,F2,11.414357,0.084225
4,NaiveSeasonal,Entite 1,F3,514.929204,0.266055
5,NaiveSeasonal,Entite 2,F3,55.606973,0.181424
6,NaiveSeasonal,Entite 5,F3,668.036178,0.241175
7,NaiveSeasonal,Entite 1,F4,6.296296,0.111555
8,NaiveSeasonal,Entite 1,F5,12.370742,0.106024
9,NaiveSeasonal,Entite 3,F5,24.071193,0.163102


In [67]:
df_merged = pd.merge(df_auto_arima, df_naive_saisonal, on=['famille', 'entite'])
df_merged = pd.merge(df_merged, df_sarima, on=['famille', 'entite'])
df_merged

,model_x,entite,famille,mape_x,rmse_x,model_y,mape_y,rmse_y,model,mape,rmse
0,NaiveDrift,Entite 1,F2,4.191600,0.041404,NaiveSeasonal,2.844508,0.033093,Sarima,6.782866,0.068331
1,NaiveDrift,Entite 2,F2,11.255404,0.063640,NaiveSeasonal,35.387882,0.145590,Sarima,25.293153,0.106689
2,NaiveDrift,Entite 3,F2,9.400653,0.099405,NaiveSeasonal,16.542492,0.191317,Sarima,13.002076,0.148313
3,NaiveDrift,Entite 5,F2,8.895829,0.065574,NaiveSeasonal,11.414357,0.084225,Sarima,8.478303,0.061541
4,NaiveDrift,Entite 1,F3,328.953915,0.157576,NaiveSeasonal,514.929204,0.266055,Sarima,390.206353,0.209340
5,NaiveDrift,Entite 2,F3,31.549814,0.143222,NaiveSeasonal,55.606973,0.181424,Sarima,45.078620,0.155179
6,NaiveDrift,Entite 5,F3,357.930763,0.213600,NaiveSeasonal,668.036178,0.241175,Sarima,380.610118,0.223811
7,NaiveDrift,Entite 1,F5,8.654989,0.069651,NaiveSeasonal,12.370742,0.106024,Sarima,11.945766,0.102069
8,NaiveDrift,Entite 3,F5,11.023372,0.078049,NaiveSeasonal,24.071193,0.163102,Sarima,24.805123,0.164567
9,NaiveDrift,Entite 4,F5,13.175096,0.104633,NaiveSeasonal,23.979418,0.188628,Sarima,20.686650,0.160283


In [89]:
df_merged['best_mape'] = ''

for i, row in df_merged[['mape', 'mape_x', 'mape_y']].iterrows():
    mape_min = 1000000
    mape_name = ''
    for col in ['mape', 'mape_x', 'mape_y']:
        if row[col] < mape_min:
            mape_min = row[col]
            mape_name = col
    df_merged.at[i, 'best_mape'] = mape_name
df_merged

,model_x,entite,famille,mape_x,rmse_x,model_y,mape_y,rmse_y,model,mape,rmse,best_mape
0,NaiveDrift,Entite 1,F2,4.191600,0.041404,NaiveSeasonal,2.844508,0.033093,Sarima,6.782866,0.068331,mape_y
1,NaiveDrift,Entite 2,F2,11.255404,0.063640,NaiveSeasonal,35.387882,0.145590,Sarima,25.293153,0.106689,mape_x
2,NaiveDrift,Entite 3,F2,9.400653,0.099405,NaiveSeasonal,16.542492,0.191317,Sarima,13.002076,0.148313,mape_x
3,NaiveDrift,Entite 5,F2,8.895829,0.065574,NaiveSeasonal,11.414357,0.084225,Sarima,8.478303,0.061541,mape
4,NaiveDrift,Entite 1,F3,328.953915,0.157576,NaiveSeasonal,514.929204,0.266055,Sarima,390.206353,0.209340,mape_x
5,NaiveDrift,Entite 2,F3,31.549814,0.143222,NaiveSeasonal,55.606973,0.181424,Sarima,45.078620,0.155179,mape_x
6,NaiveDrift,Entite 5,F3,357.930763,0.213600,NaiveSeasonal,668.036178,0.241175,Sarima,380.610118,0.223811,mape_x
7,NaiveDrift,Entite 1,F5,8.654989,0.069651,NaiveSeasonal,12.370742,0.106024,Sarima,11.945766,0.102069,mape_x
8,NaiveDrift,Entite 3,F5,11.023372,0.078049,NaiveSeasonal,24.071193,0.163102,Sarima,24.805123,0.164567,mape_x
9,NaiveDrift,Entite 4,F5,13.175096,0.104633,NaiveSeasonal,23.979418,0.188628,Sarima,20.686650,0.160283,mape_x


In [91]:
df_merged.to_csv('meilleur_modele', index=False)


In [93]:
df_sarima = pd.read_csv("sarima_parameters.csv")
df_sarima

,Entite,Famille,d,D,m,P,Q,p,q
0,Entite 1,F1,0,1,7,1,0,0,0
1,Entite 2,F1,0,1,7,1,0,1,1
2,Entite 5,F1,0,1,7,1,0,1,2
3,Entite 1,F2,1,1,7,1,0,0,1
4,Entite 2,F2,0,1,7,1,0,2,0
5,Entite 3,F2,0,1,7,1,0,0,0
6,Entite 5,F2,0,1,7,1,0,0,0
7,Entite 1,F3,0,1,7,1,0,3,1
8,Entite 2,F3,0,1,7,1,0,3,1
9,Entite 5,F3,0,1,7,1,0,3,2
